# Data 模块教程

本教程介绍 open-xquant 数据层的核心用法：**下载市场数据**和**读取市场数据**。

数据层采用 **Download-then-Read** 架构：
- **Downloader** 负责从外部 API 获取数据，保存为本地 Parquet 文件
- **LocalMarketDataProvider** 负责从本地 Parquet 文件读取数据，提供给策略管道

这种分离确保了策略执行的确定性——策略管道永远不会直接访问网络。

## 1. 安装依赖

open-xquant 的数据源作为可选依赖安装：

```bash
# 美股/全球市场（yfinance）
pip install open-xquant[yfinance]

# A 股市场（akshare）
pip install open-xquant[akshare]

# 两者都安装
pip install open-xquant[yfinance,akshare]
```

---
## 2. 下载美股数据（YFinance）

使用 `YFinanceDownloader` 从 Yahoo Finance 下载 OHLCV 日线数据。

In [ ]:
from oxq.data import YFinanceDownloader

downloader = YFinanceDownloader()

# 下载 Apple 股票 2024 年全年数据
path = downloader.download("AAPL", start="2024-01-01", end="2024-12-31")
print(f"数据已保存到: {path}")

下载完成后，数据以 Parquet 格式存储在本地。我们可以直接读取验证：

In [ ]:
import pandas as pd

df = pd.read_parquet(path)
print(f"数据形状: {df.shape}")
print(f"时间范围: {df.index[0].date()} ~ {df.index[-1].date()}")
print(f"列: {list(df.columns)}")
print()
df.head()

所有 Downloader 输出统一的 DataFrame 格式：
- **Index**: `DatetimeIndex`，名称为 `date`
- **Columns**: `open`, `high`, `low`, `close`, `volume`（全小写）

---
## 3. 下载 A 股数据（AkShare）

使用 `AkShareDownloader` 下载 A 股日线数据。注意 A 股 symbol 使用纯数字代码（如 `600519`）。

In [ ]:
from oxq.data import AkShareDownloader

ak_downloader = AkShareDownloader()

# 下载贵州茅台 2024 年数据（前复权）
path_cn = ak_downloader.download("600519", start="20240101", end="20241231")
print(f"数据已保存到: {path_cn}")

In [ ]:
df_cn = pd.read_parquet(path_cn)
print(f"数据形状: {df_cn.shape}")
print(f"时间范围: {df_cn.index[0].date()} ~ {df_cn.index[-1].date()}")
print()
df_cn.head()

无论数据源是 yfinance 还是 akshare，输出格式完全一致。这是框架的核心设计原则——**统一的数据接口**。

---
## 4. 读取数据（LocalMarketDataProvider）

下载完成后，策略通过 `LocalMarketDataProvider` 读取本地数据。这是策略管道获取行情数据的**唯一入口**。

In [ ]:
from oxq.data import LocalMarketDataProvider

provider = LocalMarketDataProvider()

# 读取指定时间范围的数据
bars = provider.get_bars("AAPL", start="2024-06-01", end="2024-06-30")
print(f"6 月数据: {len(bars)} 个交易日")
bars

`get_latest()` 返回最近一个交易日的数据（`pd.Series`）：

In [ ]:
latest = provider.get_latest("AAPL")
print(f"最新收盘价: {latest['close']}")
print(f"最新成交量: {latest['volume']:,}")
print()
latest

---
## 5. 自定义数据目录

默认情况下，数据存储在 `~/.oxq/data/market/`。有三种方式自定义路径：

### 方式一：构造参数

In [ ]:
from pathlib import Path
import tempfile

# 使用临时目录演示
custom_dir = Path(tempfile.mkdtemp()) / "my_data"

# Downloader 和 Provider 都支持自定义目录
downloader.download("MSFT", "2024-01-01", "2024-03-31", dest_dir=custom_dir)

custom_provider = LocalMarketDataProvider(data_dir=custom_dir)
msft = custom_provider.get_bars("MSFT", "2024-01-01", "2024-03-31")
print(f"自定义目录读取成功: {len(msft)} 条数据")
print(f"数据目录: {custom_dir}")

### 方式二：环境变量

设置 `OXQ_DATA_DIR` 环境变量，所有组件自动使用 `$OXQ_DATA_DIR/market/` 作为数据目录：

```bash
export OXQ_DATA_DIR=/path/to/my/data
```

### 优先级

构造参数 > 环境变量 > 默认值 (`~/.oxq/data/market/`)

---
## 6. 错误处理

数据层提供明确的异常类型，方便调用方处理。

In [ ]:
from oxq.core import SymbolNotFoundError, DownloadError

# 读取未下载的标的 → SymbolNotFoundError
try:
    provider.get_bars("UNKNOWN_SYMBOL", "2024-01-01", "2024-12-31")
except SymbolNotFoundError as e:
    print(f"捕获异常: {e}")

In [ ]:
# 下载不存在的标的 → DownloadError
try:
    downloader.download("INVALID_TICKER_XYZ", "2024-01-01", "2024-12-31")
except DownloadError as e:
    print(f"捕获异常: {e}")

两种异常都继承自 `OxqError`，可以统一捕获：

```python
from oxq.core import OxqError

try:
    ...
except OxqError as e:
    print(f"open-xquant 错误: {e}")
```

---
## 7. 批量下载

使用 `download_many()` 一次下载多个标的：

In [ ]:
symbols = ["GOOGL", "AMZN", "META"]
paths = downloader.download_many(symbols, start="2024-01-01", end="2024-12-31")

for symbol, path in paths.items():
    df = pd.read_parquet(path)
    print(f"{symbol}: {len(df)} 个交易日, 最新收盘价 {df['close'].iloc[-1]:.2f}")

下载完成后，可以通过 `LocalMarketDataProvider` 统一读取所有标的：

In [ ]:
for symbol in symbols:
    latest = provider.get_latest(symbol)
    print(f"{symbol}: close={latest['close']:.2f}, volume={latest['volume']:,}")

---
## 小结

本教程覆盖了 data 模块的核心用法：

| 组件 | 职责 |
|------|------|
| `YFinanceDownloader` | 下载美股/全球市场数据 |
| `AkShareDownloader` | 下载 A 股数据 |
| `LocalMarketDataProvider` | 从本地 Parquet 读取数据 |
| `resolve_data_dir()` | 解析数据目录路径 |

**核心设计原则**：
- 下载与读取分离，策略执行不依赖网络
- 统一的 DataFrame 格式（OHLCV + DatetimeIndex）
- Protocol 接口，支持自定义数据源扩展